In [1]:
from cut import main_cut


# step 1 :cut your image

In [9]:
!rm -rf /xuhan/data0903/0903_xuhan/img_cut/
!mkdir /xuhan/data0903/0903_xuhan/img_cut/

import os
if __name__ == '__main__':
    img_dir = r'/xuhan/data0903/0903_xuhan/xuhan/'
    os.chdir(img_dir)
    save_dir = r'/xuhan/data0903/0903_xuhan/img_cut/' # os.path.join(img_dir, 'processed')
    main_cut(img_dir, save_dir, num_hor_pieces = 12, num_ver_pieces = 6, flag = 'jk_cengqian', mode = 'cut')


# step 2 : use model to predict and gen csv

In [1]:
import glob
data = glob.glob('/xuan/data/xuhan_calssify/modelData8/img/*.jpg')
len(data)


2891

In [4]:

!rm -rf /xuhan/data0903/0906_xuhan/img/
!rm -rf /xuhan/data0903/0906_xuhan/xml/
!rm -rf /xuhan/data0903/0906_xuhan/output/


!mkdir /xuhan/data0903/0906_xuhan/img/
!mkdir /xuhan/data0903/0906_xuhan/xml/
!mkdir /xuhan/data0903/0906_xuhan/output/


!python2 /xuhan/genData/infer_simple.py \
--cfg /xuhan/data_0905/mixedData/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml \
--output-dir /xuhan/data0903/0906_xuhan/output/  \
--image-ext jpg  \
--threshold 0.90 \
--img-save /xuhan/data0903/0906_xuhan/img/ \
--wts /xuhan/data_0905/mixedData/output/train/voc_2007_train/generalized_rcnn/model_final.pkl \
--im_or_folder /xuan/data/xuhan_calssify/modelData8/img/

# --im_or_folder /xuhan/data0903/0906_xuhan/img_cut/


Found Detectron ops lib: /usr/local/caffe2_build/lib/libcaffe2_detectron_ops_gpu.so
E0906 10:09:04.709204   204 init_intrinsics_check.cc:54] CPU feature avx is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
E0906 10:09:04.709230   204 init_intrinsics_check.cc:54] CPU feature avx2 is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
E0906 10:09:04.709234   204 init_intrinsics_check.cc:54] CPU feature fma is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
WARNING cnn.py:  40: [====DEPRECATE WARNING====]: you are creating an object from CNNModelHelper class which will be deprecated soon. Please use ModelHelper object with brew module. For more information, please refer to caffe2.ai and python/brew.py, python/brew_test.py for more information.
INFO net.py:  57: Lo

In [5]:
import pandas as pd
from pandas.core.frame import DataFrame
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt
data  = pd.read_csv('../data0906/result.csv')
print(len(data))
# data.loc[data.filename == '13C5A1180731110253372047_1x3.jpg',:]

3116


# step 3 : genXML


In [6]:
!rm -rf  /xuhan/data0903/0906_xuhan/xml
!mkdir /xuhan/data0903/0906_xuhan/xml
import cv2
import os


# subdata.loc[:,['xmin','ymin','xmax','ymax']] = subdata.loc[:,['xmin','ymin','xmax','ymax']].astype(int)
for filename ,subdata in data.groupby('filename'):
    img = cv2.imread( '/xuhan/data0903/0906_xuhan/img/' + filename )
    #print(img.shape)
    subdata = subdata.copy()
    subdata.loc[:,['xmin','ymin','xmax','ymax']] = subdata.loc[:,['xmin','ymin','xmax','ymax']].astype(int)
    temp = ET.parse('./template.xml')
    root = temp.getroot()

    # modify file name
    file = root.find('filename')
    file.text =  filename

    # modify path
    path = root.find("path")

    path.text = filename

    # modify size
    width = root.find('.//size//width')
    width.text = str(img.shape[1])
    height = root.find('.//size//height')
    height.text = str(img.shape[0])
    depth = root.find('.//size//depth')
    depth.text = str(img.shape[2])
    for idx,bbox_info in  subdata.iterrows():

        b = ET.SubElement(root, 'object')
        name = ET.SubElement(b, 'name')
        pose = ET.SubElement(b, 'pose')
        bndbox = ET.SubElement(b, 'bndbox')
        xmin = ET.SubElement(bndbox, 'xmin')
        ymin = ET.SubElement(bndbox, 'ymin')
        xmax = ET.SubElement(bndbox, 'xmax')
        ymax = ET.SubElement(bndbox, 'ymax')

        # 修改
        name.text = str(bbox_info['class'])
        pose.text = 'Unspecified'
        xmin.text = str(bbox_info['xmin'])
        ymin.text = str(bbox_info['ymin'])
        xmax.text = str(bbox_info['xmax'])
        ymax.text = str(bbox_info['ymax'])
    #print('/xuhan/data0903/0903_xuhan/xml/'+ os.path.basename(filename).split('.')[0] +  '.xml')
    temp.write('/xuhan/data0903/0906_xuhan/xml/'+ os.path.basename(filename).split('.')[0] +  '.xml')

#     break